In [51]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [52]:
path = '/Users/alexandra/Desktop/ML research/'

In [61]:
y = pd.read_excel(path + 'features.xlsx', usecols=['survey_reliance'])
x = pd.read_excel(path + 'features.xlsx', 
                  usecols=[
                           'total_duration',
                           'percent_gaze_duration_on_ai',
                           #'total_duration_on_ai', *
                            
                           #check if those outputs could be used
                           'Trust',
                           'Familiarity',
                           'Age',
                           'Education',
                           'Gender',
                           
                           #try to include gender 
                           #exluded the not impacting features 
                           #use some process? below a threshold, exclude it ->pick a good paper's value
                           #k-squre likelihood ratio test 
                           #nova between model has a feature and without feature
                           
                           
                           'percent_duration_task',
                           #'total_duration_on_task',
                           'percent_duration_user', 
                           'total_duration_on_user', 
                           'first_gaze_on_ai_start_time', 
                           'first_gaze_on_ai_duration',
                           #'count_ai2task', 
                           'percent_shift_ai2task', 
                           #'count_ai2user',
                           'percent_shift_ai2user',                       
                           #'count_task2user', 
                           'percent_shift_task2user',
                           #'count_task2ai', 
                           #'count_2task',
                           'percent_shift_task2ai',
                           #'count_user2ai',
                           'percent_shift_2ai',                            
                           'percent_shift_user2ai',
                           #'count_user2task', 
                           #'percent_shift_user2task'* -> constant,
                           'percent_shift_2user', 
                           #'count_2ai',  *
                           'count_2user',
                           'start_user_ai_acc',
                           #'Orientation'   
                           ])

#x['Orientation'] = x['Orientation'].replace([1],[5.0])


In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.11, shuffle = False)

y_train = y_train.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
y_train = y_train.astype(cat_type)

#y_test = y_test.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
#cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
#y_test = y_test.astype(cat_type)

y = y.replace([1, 2, 3, 4, 5, 6, 7], ['A', 'B', 'C', 'D', 'E', 'F', 'G'])
cat_type = CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G'], ordered=True)
y = y.astype(cat_type)

In [63]:
mod_prob = OrderedModel(y['survey_reliance'], x, distr='probit')
res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 1.646402
         Iterations: 135
         Function evaluations: 139
         Gradient evaluations: 139


/Users/alexandra/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:        survey_reliance   Log-Likelihood:                -563.07
Model:                   OrderedModel   AIC:                             1180.
Method:            Maximum Likelihood   BIC:                             1284.
Date:                Tue, 17 Jan 2023                                         
Time:                        01:40:35                                         
No. Observations:                 342                                         
Df Residuals:                     315                                         
Df Model:                          27                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
total_duration                 -0.0006        nan        nan        nan         nan         nan
percent_gaze_duration_on_ai    -1.4579        nan        nan        nan         nan         nan
Age                            -0.0053        nan        nan        nan         nan         nan
Gender                          0.0732        nan        nan        nan         nan         nan
Education                       0.0239        nan        nan        nan         nan         nan
Familiarity                    -0.1607        nan        nan        nan         nan         nan
Trust                          -0.1419        nan        nan        nan         nan         nan
percent_duration_task           0.3303        nan        nan        nan         nan         nan
percent_duration_user           2.2462        nan        nan        nan         nan         nan
total_duration_on_user          0.0271        nan        nan        nan         nan         nan
first_gaze_on_ai_start_time     0.0022        nan        nan        nan         nan         nan
first_gaze_on_ai_duration      -0.2477        nan        nan        nan         nan         nan
percent_shift_ai2task           7.3110        nan        nan        nan         nan         nan
percent_shift_ai2user           3.6539        nan        nan        nan         nan         nan
percent_shift_task2user        -3.6390        nan        nan        nan         nan         nan
percent_shift_task2ai          -0.8805        nan        nan        nan         nan         nan
percent_shift_user2ai           0.9179        nan        nan        nan         nan         nan
percent_shift_2ai               0.0374        nan        nan        nan         nan         nan
percent_shift_2user             0.0149        nan        nan        nan         nan         nan
count_2user                    -0.0091        nan        nan        nan         nan         nan
start_user_ai_acc               0.4961        nan        nan        nan         nan         nan
A/B                            -1.4682        nan        nan        nan         nan         nan
B/C                             0.0569        nan        nan        nan         nan         nan
C/D                            -1.2275        nan        nan        nan         nan         nan
D/E                            -2.2690        nan        nan        nan         nan         nan
E/F                            -0.5587        nan        nan        nan         nan         nan
F/G                            -0.2974        nan        nan        nan         nan         nan
===============================================================================================
"""

In [64]:
y_hat = res_prob.predict(x_test)
value = y_hat.to_numpy()

def expected_value(values, weights):
    values = np.asarray(values)
    weights = np.asarray(weights)
    return (values * weights).sum() 


EV = []
weights = [1,2,3,4,5,6,7]

for i in range(0,38,1):
    values = value[i]
    ev = expected_value(values,weights);
    EV.append(ev)


r_square = mean_squared_error(y_test, EV)
r_square

2.0454635299888166